In [84]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge, HuberRegressor, LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
import catboost as cb
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import PolynomialFeatures

In [48]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [49]:
print("Null columns in train dataset")
for col in df.columns:
    null = df[col].isnull().sum()
    if null != 0:
        print(df[col].dtypes,col, null)

Null columns in train dataset
float64 LotFrontage 259
object Alley 1369
object MasVnrType 8
float64 MasVnrArea 8
object BsmtQual 37
object BsmtCond 37
object BsmtExposure 38
object BsmtFinType1 37
object BsmtFinType2 38
object Electrical 1
object FireplaceQu 690
object GarageType 81
float64 GarageYrBlt 81
object GarageFinish 81
object GarageQual 81
object GarageCond 81
object PoolQC 1453
object Fence 1179
object MiscFeature 1406


In [50]:
print("Null columns in test dataset")
for col in test.columns:
    null = test[col].isnull().sum()
    if null != 0:
        print(test[col].dtypes, col, null)

Null columns in test dataset
object MSZoning 4
float64 LotFrontage 227
object Alley 1352
object Utilities 2
object Exterior1st 1
object Exterior2nd 1
object MasVnrType 16
float64 MasVnrArea 15
object BsmtQual 44
object BsmtCond 45
object BsmtExposure 44
object BsmtFinType1 42
float64 BsmtFinSF1 1
object BsmtFinType2 42
float64 BsmtFinSF2 1
float64 BsmtUnfSF 1
float64 TotalBsmtSF 1
float64 BsmtFullBath 2
float64 BsmtHalfBath 2
object KitchenQual 1
object Functional 2
object FireplaceQu 730
object GarageType 76
float64 GarageYrBlt 78
object GarageFinish 78
float64 GarageCars 1
float64 GarageArea 1
object GarageQual 78
object GarageCond 78
object PoolQC 1456
object Fence 1169
object MiscFeature 1408
object SaleType 1


In [51]:
y = df['SalePrice']
df = df.drop(['SalePrice'],axis=1)
df = df.set_index('Id')
test = test.set_index('Id')

In [52]:
a = pd.concat([df,test],axis=0)
df = a.reset_index(drop=True)

In [53]:
# Get list of categorical variables in holiday dataset
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print("\n Categorical variables:")
print(object_cols)
for col in df[object_cols]:
        print("\n"+col)
        print('Number of categories: '+str(len(df[col].unique())))
        print(df[col].unique())
            


 Categorical variables:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

MSZoning
Number of categories: 6
['RL' 'RM' 'C (all)' 'FV' 'RH' nan]

Street
Number of categories: 2
['Pave' 'Grvl']

Alley
Number of categories: 3
[nan 'Grvl' 'Pave']

LotShape
Number of categories: 4
['Reg' 'IR1' 'IR2' 'IR3']

LandContour
Number of categories: 4
['Lvl' 'Bnk' 'Low' 'HLS']

Utilities
Number of categories: 3
['AllPub' 'NoSeWa' nan]

LotConfig
Number of categories: 5
['Inside'

In [54]:
low_cardinality_cols=[]
med_cardinality_cols=[]
for col in df[object_cols]:
    if len(df[col].unique())<=10:
        low_cardinality_cols.append(col)
    elif len(df[col].unique())>10:
        med_cardinality_cols.append(col)

In [55]:
for col in df[df.columns]:
    if col not in object_cols:
        scaler = MinMaxScaler()
        df[col] = scaler.fit_transform(df[[col]])

In [56]:
# One Hot Encoding for Low Cardinality Columns
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(df[low_cardinality_cols]))
OH_cols.index = df.index
OH_cols.columns = OH_encoder.get_feature_names(low_cardinality_cols)
df = df.drop(low_cardinality_cols, axis=1)
df = pd.concat([df, OH_cols], axis=1)

# Ordinal Encoding for Medium Cardinality Columns
label_encoder = OrdinalEncoder()
df[med_cardinality_cols] = label_encoder.fit_transform(df[med_cardinality_cols])

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [57]:
for col in df.columns:
    null = df[col].isnull().sum()
    if null != 0:
        df = df[(df[col].notna()) | (df.index >= 1460)]
        print(df[col].dtypes, col, null)

float64 LotFrontage 486
float64 Exterior1st 1
float64 Exterior2nd 1
float64 MasVnrArea 21
float64 BsmtFinSF1 1
float64 BsmtFinSF2 1
float64 BsmtUnfSF 1
float64 TotalBsmtSF 1
float64 BsmtFullBath 2
float64 BsmtHalfBath 2
float64 GarageYrBlt 152
float64 GarageCars 1
float64 GarageArea 1


In [58]:
df = df.fillna(0)

In [110]:
df_X = df[df.index < 1460]
test = df[df.index >= 1460]
full = pd.merge(left = df_X, right = y , left_index= True, right_index = True)
train, dev = train_test_split(full, test_size=0.2)

train_y = np.log(train['SalePrice'])
train_X = train.drop(['SalePrice'],axis=1)

dev_y = np.log(dev['SalePrice'])
dev_X = dev.drop(['SalePrice'],axis=1)

In [208]:
# rfmodel = RandomForestRegressor(n_jobs=-1)
# rfparams = {
#     'max_depth' : [26],
#     'n_estimators' : [1000],
#     'min_samples_leaf':[4],
#     'max_features': ['auto'],
#     'oob_score' : [True],
#     'min_samples_split' : [4],
#     'ccp_alpha' : [0.00005],
#     'max_samples' : [0.65]
# }
# rfgrid = GridSearchCV(estimator = rfmodel,verbose=4, param_grid = rfparams, cv = 10, scoring = 'neg_root_mean_squared_error')
# rfgrid.fit(train_X,train_y)
# print(rfgrid.best_params_)
# print(rfgrid.best_score_)
# print(rfgrid.score(dev_X,dev_y))
# print(rfgrid.score(train_X,train_y))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END ccp_alpha=6e-05, max_depth=26, max_features=auto, max_samples=0.6, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, oob_score=True;, score=-0.127 total time=   4.6s
[CV 2/10] END ccp_alpha=6e-05, max_depth=26, max_features=auto, max_samples=0.6, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, oob_score=True;, score=-0.135 total time=   4.0s
[CV 3/10] END ccp_alpha=6e-05, max_depth=26, max_features=auto, max_samples=0.6, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, oob_score=True;, score=-0.148 total time=   4.2s
[CV 4/10] END ccp_alpha=6e-05, max_depth=26, max_features=auto, max_samples=0.6, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, oob_score=True;, score=-0.138 total time=   4.2s
[CV 5/10] END ccp_alpha=6e-05, max_depth=26, max_features=auto, max_samples=0.6, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, oob_score=True;, score=-0.139 total ti

In [247]:
rfmodel = RandomForestRegressor(ccp_alpha= 0.00005, max_depth=26, max_features= 'auto', max_samples=0.6, min_samples_leaf= 4, min_samples_split= 4, n_estimators= 1000, oob_score= True)
rfmodel.fit(train_X,train_y)
rfpred = rfmodel.predict(dev_X)

In [203]:
# xgbmodel = XGBRegressor(n_jobs=-1)
# xgbparams = {
#     'max_depth' : [3],
#     'n_estimators' : [10000],
#     'learning_rate' : [0.01],
#     'colsample_bytree' : [0.6],
#     'subsample':[0.65],
#     'alpha' : [3],
#     'lambda' : [3],
# }
# xgbgrid = GridSearchCV(estimator = xgbmodel,verbose=3, param_grid = xgbparams, cv = 5, scoring = 'neg_root_mean_squared_error')
# xgbgrid.fit(train_X,train_y)
# print(xgbgrid.best_params_)
# print(xgbgrid.best_score_)
# print(xgbgrid.score(dev_X,dev_y))
# print(xgbgrid.score(train_X,train_y))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 1/5] END alpha=3, colsample_bytree=0.6, lambda=3, learning_rate=0.01, max_depth=3, n_estimators=10000, subsample=0.65;, score=-0.112 total time=  46.1s


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 2/5] END alpha=3, colsample_bytree=0.6, lambda=3, learning_rate=0.01, max_depth=3, n_estimators=10000, subsample=0.65;, score=-0.133 total time=  45.8s


...

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 5/5] END alpha=3, colsample_bytree=0.6, lambda=3, learning_rate=0.01, max_depth=3, n_estimators=10000, subsample=0.65;, score=-0.144 total time=  43.3s


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


{'alpha': 3, 'colsample_bytree': 0.6, 'lambda': 3, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 10000, 'subsample': 0.65}
-0.13346724426465828
-0.11465244678030788
-0.09099190457269914


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [248]:
xgbmodel = XGBRegressor(alpha= 3, colsample_bytree=0.6, reg_lambda=3, learning_rate= 0.01, max_depth=3, n_estimators=10000, subsample=0.65)
xgbmodel.fit(train_X,train_y)
xgbpred = xgbmodel.predict(dev_X)

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [168]:
# ridgemodel = Ridge()
# ridgeparams = {
#     'alpha' : [3]
# }
# ridgegrid = GridSearchCV(estimator = ridgemodel,verbose=3, param_grid = ridgeparams, cv = 10, scoring = 'neg_root_mean_squared_error')
# ridgegrid.fit(train_X,train_y)
# print(ridgegrid.best_params_)
# print(ridgegrid.best_score_)
# print(ridgegrid.score(dev_X,dev_y))
# print(ridgegrid.score(train_X,train_y))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .........................alpha=3;, score=-0.136 total time=   0.0s
[CV 2/10] END .........................alpha=3;, score=-0.132 total time=   0.1s
[CV 3/10] END .........................alpha=3;, score=-0.153 total time=   0.0s
[CV 4/10] END .........................alpha=3;, score=-0.127 total time=   0.0s
[CV 5/10] END .........................alpha=3;, score=-0.142 total time=   0.0s
[CV 6/10] END .........................alpha=3;, score=-0.212 total time=   0.0s
[CV 7/10] END .........................alpha=3;, score=-0.111 total time=   0.0s
[CV 8/10] END .........................alpha=3;, score=-0.133 total time=   0.0s
[CV 9/10] END .........................alpha=3;, score=-0.122 total time=   0.0s
[CV 10/10] END ........................alpha=3;, score=-0.263 total time=   0.0s
{'alpha': 3}
-0.1531681891250692
-0.12615726835395794
-0.11641913685469803


In [249]:
ridgemodel = Ridge(alpha=3)
ridgemodel.fit(train_X,train_y)
ridgepred = ridgemodel.predict(dev_X)

In [205]:
# svrmodel = SVR()
# svrparams = {
#     'C' : [35],
#     'epsilon' : [0.008],
#     'gamma' : [0.0008]
# }
# svrgrid = GridSearchCV(estimator = svrmodel,verbose=3, param_grid = svrparams, cv = 10, scoring = 'neg_root_mean_squared_error')
# svrgrid.fit(train_X,train_y)
# print(svrgrid.best_params_)
# print(svrgrid.best_score_)
# print(svrgrid.score(dev_X,dev_y))
# print(svrgrid.score(train_X,train_y))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.125 total time=   0.2s
[CV 2/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.121 total time=   0.2s
[CV 3/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.147 total time=   0.3s
[CV 4/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.128 total time=   0.3s
[CV 5/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.131 total time=   0.3s
[CV 6/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.204 total time=   0.2s
[CV 7/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.099 total time=   0.3s
[CV 8/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.128 total time=   0.3s
[CV 9/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.105 total time=   0.2s
[CV 10/10] END C=35, epsilon=0.008, gamma=0.0008;, score=-0.274 total time=   0.2s
{'C': 35, 'epsilon': 0.008, 'gamma': 0.0008}
-0.14632329321956797
-0.12081510562302687
-0.1181902029633332


In [250]:
svrmodel = SVR(C=35,epsilon=0.008,gamma=0.0008)
svrmodel.fit(train_X,train_y)
svrpred = svrmodel.predict(dev_X)

In [209]:
# hubermodel = HuberRegressor()
# huberparams = {
#     'epsilon' : [5],
#     'alpha' : [20],
#     'max_iter' : [600],
#     'fit_intercept' : [True]
# }
# hubergrid = GridSearchCV(estimator = hubermodel,verbose=3, param_grid = huberparams, cv = 10, scoring = 'neg_root_mean_squared_error')
# hubergrid.fit(train_X,train_y)
# print(hubergrid.best_params_)
# print(hubergrid.best_score_)
# print(hubergrid.score(dev_X,dev_y))
# print(hubergrid.score(train_X,train_y))

Fitting 10 folds for each of 27 candidates, totalling 270 fits


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 1/10] END alpha=20, epsilon=5, fit_intercept=True, max_iter=500;, score=-0.135 total time=   0.8s


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 2/10] END alpha=20, epsilon=5, fit_intercept=True, max_iter=500;, score=-0.125 total time=   1.1s


...

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 9/10] END alpha=30, epsilon=9, fit_intercept=True, max_iter=700;, score=-0.122 total time=   0.8s


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 10/10] END alpha=30, epsilon=9, fit_intercept=True, max_iter=700;, score=-0.261 total time=   0.8s
{'alpha': 20, 'epsilon': 5, 'fit_intercept': True, 'max_iter': 600}
-0.14957729959134566
-0.12256167073892144
-0.11557586044977412


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [251]:
hubermodel = HuberRegressor(alpha=20,epsilon=5,fit_intercept=True,max_iter=600)
hubermodel.fit(train_X,train_y)
huberpred = hubermodel.predict(dev_X)

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
lightgbm = LGBMRegressor(objective='regression')
lightgbmparams = {
    'learning_rate'=[0.01], 
    'n_estimators' = [2000],
    'lambda_l1' : [],
    'lambda_l2' : [],
    'max_depth' : [],
}
lightgbmgrid = GridSearchCV(estimator = lightgbm,verbose=3, param_grid = lightgbmparams, cv = 10, scoring = 'neg_root_mean_squared_error')
lightgbmgrid.fit(train_X,train_y)
print(lightgbmgrid.best_params_)
print(lightgbmgrid.best_score_)
print(lightgbmgrid.score(dev_X,dev_y))
print(lightgbmgrid.score(train_X,train_y))

In [17]:
lightgbm = LGBMRegressor(objective='regression', learning_rate=0.01, n_estimators=2000)
lightgbm.fit(train_X,train_y)
lightpred = lightgbm.predict(dev_X)

0.12391225415241319
0.02781533222536997


In [ ]:
cbmodel = cb.CatBoostRegressor(loss_function='RMSE',verbose=False)
cbparams = {
    'learning_rate'=[0.01], 
    'n_estimators' = [1000],

}
cbgrid = GridSearchCV(estimator = cbmodel,verbose=3, param_grid = cbparams, cv = 10, scoring = 'neg_root_mean_squared_error')
cbgrid.fit(train_X,train_y)
print(cbgrid.best_params_)
print(cbgrid.best_score_)
print(cbgrid.score(dev_X,dev_y))
print(cbgrid.score(train_X,train_y))

In [219]:
cbmodel = cb.CatBoostRegressor(loss_function='RMSE',learning_rate=0.01, n_estimators=1000,verbose=False)
cbmodel.fit(train_X, train_y)
cbpred = cbmodel.predict(dev_X)
cbfit = cbmodel.predict(train_X)

print(mean_squared_error(dev_y,cbpred, squared=False))
print(mean_squared_error(train_y,cbfit, squared=False))

0.11561257925834084
0.08292366587438614


In [243]:
stackmodel = StackingCVRegressor(regressors=(ridgemodel, xgbmodel, rfmodel, hubermodel, cbmodel),
                                meta_regressor=xgbmodel,
                                use_features_in_secondary=True)
stackmodel.fit(np.array(train_X), np.array(train_y))
stackpred = stackmodel.predict(np.array(dev_X))
stackfit = stackmodel.predict(np.array(train_X))

print(mean_squared_error(dev_y,stackpred, squared=False))
print(mean_squared_error(train_y,stackfit, squared=False))

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to conv

0.11149721276761093
0.08256772757832642


In [252]:
pred = (rfpred + xgbmodel.predict(dev_X) +  stackmodel.predict(dev_X) + ridgepred + huberpred + cbpred) / 6
print(mean_squared_error(dev_y,pred, squared=False))

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature nam

0.10941402808001804


/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but HuberRegressor was fitted without feature names
  warnings.warn(


In [253]:
final_prediction = (np.exp(ridgemodel.predict(test))+ np.exp(xgmodel.predict(test)) \
 + np.exp(rfmodel.predict(test)) + np.exp(stackmodel.predict(test)) \
 + np.exp(hubermodel.predict(test)) + np.exp(cbmodel.predict(test)))/6 

/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/wee/Desktop/Github/store_sales_forecast/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature nam

In [254]:
submission = pd.DataFrame(final_prediction, index = test.index)

In [255]:
submission.reset_index(drop=False, inplace = True)
submission = submission.rename(columns={0 : 'SalePrice', 'index' : 'Id'})
submission.to_csv('submission_v2.csv', index=False)

In [256]:
len(submission)

1459